In [1]:
# Importing packages
import numpy as np
from scipy.optimize import basinhopping
import numba as nb

# Importing figure properties
from figures_report.figure_properties import *

# Importing functions
from hamiltonian.hamiltonian import get_hamiltonian
from Bell_operator.Bell_operator import calc_Bell_operator
from classical_optimization.classical_bound import calc_classical_bound

# Defining saving directory
save_dir = '../data/figures/hydrogen_mol/'
save_fdir = '../data/data_sets/hydrogen_mol/'

In [2]:
# Set parameters to make a simple molecule.
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., .7414))] # in Angstroms
basis = 'sto-3G'
multiplicity = 1
charge = 0

# Defining qubit transform
qubit_transform = 'JW'

# Obtainig Hamiltonian
hamiltonian = get_hamiltonian(geometry, basis, multiplicity, charge, qubit_transform = qubit_transform)

# Extracting parameters
H = hamiltonian.matrix_form
N = hamiltonian.N

# Calculating the eigenvalues and eigenstates
eig_vals, eig_vecs = np.linalg.eigh(H)

# Extracting the ground state energy and the respective eigenstate
psi_G = np.zeros((2**N, 1))
psi_G[:,0] = eig_vecs[:,0]
rho = np.matmul(psi_G, np.transpose(psi_G))


In [5]:
# Defining small error
eps = 1e-3

# Defining number of measurements and the angle between the two parties
m = nb.typed.List(
    [2,2,2,2]
)
theta = np.random.rand(N-1)*(np.pi-2*eps)+eps

# # Initializing basis and extra Z gate
basis = 'XY'
extra_gate = True

# Calculating the classical bound
beta_C = calc_classical_bound(theta, H, N, m, basis, extra_gate)
print('Classical bound: %.16f' %(beta_C))

# Calculating eigenvalue of the Bell operator
beta_Q = calc_Bell_operator(theta, H, rho, N, m, basis, extra_gate)
print('Quantum bound: %.16f' %(beta_Q))


Classical bound: -1.9597878527363504
Quantum bound: -1.8510241683485202


In [4]:
# Defining a fitting function and a callback function
def fitting_func(theta, H, N, m, basis, extra_Z_gate):

    return -calc_classical_bound(theta, H, N, m, basis, extra_Z_gate)

def callback_function(theta):

    stored_angles.append( theta )
    
    beta_C.append( calc_classical_bound(theta, H, N, m, basis, extra_Z_gate) )

In [ ]:
# Set parameters to make a simple molecule.
diatomic_bond_length = .7414   # Angstroms
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3G'
multiplicity = 1
charge = 0

# Defining qubit transform
qubit_transform = 'BK'

# Obtainig Hamiltonian
hamiltonian = get_hamiltonian(geometry, basis, multiplicity, charge, qubit_transform = qubit_transform)

# Extracting parameters
H = hamiltonian.matrix_form
N = hamiltonian.N

# Calculating the eigenvalues and eigenstates
eig_vals, eig_vecs = np.linalg.eigh(H)

# Extracting the ground state energy and the respective eigenstate
psi_G = np.zeros((2**N, 1))
psi_G[:,0] = eig_vecs[:,0]
rho = np.matmul(psi_G, np.transpose(psi_G))



# Calculting the quantum# Defining the number of measurements
m = nb.typed.List(
        [4,3,2,2]
)

# Initializing empty list beta_C for call_back function
beta_C = []
stored_angles = []

# Defining angles
eps = 1e-2
init_guess = np.random.rand(N-1)*(np.pi-2*eps)+eps

# Defining arguments
basis = 'XY'
extra_Z_gate = True
args = (H, N, m, basis, extra_Z_gate)

# Defining minimization options
options = {
        'maxiter'  : 2000,
}


# Compiling the fitting functions
beta_C.append( -fitting_func(init_guess, *args) )

# Defining the minimizer keyword arguments
minimizer_kwargs = {
                'method'  : "Nelder-Mead",
                'args'    : args,
                'options' : options,
                'callback': callback_function
}

res = basinhopping(fitting_func, init_guess,
                minimizer_kwargs = minimizer_kwargs,
                niter = 4,
                disp = True 
)

np.save(save_fdir+'beta_C_'+qubit_transform+'_m_%.0f.npy' %(np.sum(m)), beta_C)
np.save(save_fdir+'theta_'+qubit_transform+'_m_%.0f.npy' %(np.sum(m)), stored_angles)